In [1]:
# Import necessary Python libraries
# - pandas: for general-purpose data analysis
# - psycopg2: for connecting and querying a PostgreSQL database
# - polars: a high-performance DataFrame library used here for efficient CSV manipulation
import pandas as pd
import psycopg2
import polars as pl  # one additional library not covered in class related to data engineering

In [2]:
# Load the raw datasets using two different libraries
# - parks_df is loaded with Polars for faster string and column manipulation
# - observations_df and species_info_df are loaded with Pandas (standard in the bootcamp)
parks_df = pl.read_csv('resources/parks.csv')  # Original parks data (includes "Bryce Canyon National Park")
observations_df = pd.read_csv('resources/observations.csv')  # Animal sightings per park
species_info_df = pd.read_csv('resources/species_info.csv')  # Species category and conservation status

In [3]:
# Fix naming inconsistency:
# Replace all instances of "Bryce Canyon National Park" with "Bryce National Park" in the 'Park Name' column
# This ensures it matches the naming convention used in the observations dataset (important for joining later)
parks_df = parks_df.with_columns(
    pl.col("Park Name").str.replace_all("Bryce Canyon National Park", "Bryce National Park").alias("Park Name")
)

In [4]:
# Standardize column names to align with other datasets
# Rename 'Park Name' -> 'park_name' and 'Park Code' -> 'park_code'
# This simplifies merging by ensuring columns have consistent names across DataFrames
parks_df = parks_df.rename({
    "Park Name": "park_name",
    "Park Code": "park_code"
})

In [5]:
# Save the cleaned and standardized parks dataset to a new file called 'parks_updated.csv'
# This will be used later when merging or importing into a SQL database
parks_df.write_csv("resources/parks_updated.csv")

In [6]:
# Connect to the PostgreSQL database you previously created in pgAdmin
# This allows you to query data stored in your 'national_park_species_db' from Jupyter
conn = psycopg2.connect(
    host = "localhost",
    user = "postgres",
    password = "postgres",
    dbname = "national_park_species_db",
    port = 5432)

In [7]:
# Create a cursor object that lets you execute SQL commands from Python
cur = conn.cursor()

In [8]:
# Execute SQL to retrieve all rows from the 'observations' table
cur.execute("select * from observations")
observations = cur.fetchall()  # Store results as a list of tuples

In [9]:
# Execute SQL to retrieve all rows from the 'parks_updated' table
cur.execute("select * from parks_updated")
parks_updated = cur.fetchall()

In [10]:
# Execute SQL to retrieve all rows from the 'species_info' table
cur.execute("select * from species_info")
species_info = cur.fetchall()

In [11]:
# Print the first 3 rows of each table to visually inspect and confirm data was pulled successfully
print("Observations (first 3 rows):", observations[:3])
print("Parks Updated (first 3 rows):", parks_updated[:3])
print("Species Info (first 3 rows):", species_info[:3])

Observations (first 3 rows): [('Vicia benghalensis', 'Great Smoky Mountains National Park', 68), ('Neovison vison', 'Great Smoky Mountains National Park', 77), ('Prunus subcordata', 'Yosemite National Park', 138)]
Parks Updated (first 3 rows): [('ACAD', 'Acadia National Park', 'ME', 47390, 44.35, -68.21), ('ARCH', 'Arches National Park', 'UT', 76519, 38.68, -109.57), ('BADL', 'Badlands National Park', 'SD', 242756, 43.75, -102.5)]
Species Info (first 3 rows): [('Mammal', 'Clethrionomys gapperi gapperi', "Gapper's Red-Backed Vole", None), ('Mammal', 'Bos bison', 'American Bison, Bison', None), ('Mammal', 'Bos taurus', 'Aurochs, Aurochs, Domestic Cattle (Feral), Domesticated Cattle', None)]


In [12]:
# Perform LEFT JOINs using SQL directly from Python:
# 1. Join 'observations' with 'parks_updated' on 'park_name'
# 2. Then join that result with 'species_info' on 'scientific_name'
# This creates one unified dataset containing species observations + park details + species metadata
query = """
    SELECT o.*, 
           p.park_code, p.State, p.Acres, p.Latitude, p.Longitude,
           s.category, s.common_names, s.conservation_status
    FROM observations o
    LEFT JOIN parks_updated p ON o.park_name = p.park_name
    LEFT JOIN species_info s ON o.scientific_name = s.scientific_name
"""
cur.execute(query)
final_joined_data = cur.fetchall()  # Store the final joined result

In [13]:
# Preview a few rows from the final joined dataset to validate merge success
for row in final_joined_data[:3]:
    print(row)

('Vicia benghalensis', 'Great Smoky Mountains National Park', 68, 'GRSM', 'TN, NC', 521490, 35.68, -83.53, 'Vascular Plant', 'Purple Vetch, Reddish Tufted Vetch', None)
('Neovison vison', 'Great Smoky Mountains National Park', 77, 'GRSM', 'TN, NC', 521490, 35.68, -83.53, 'Mammal', 'American Mink', None)
('Prunus subcordata', 'Yosemite National Park', 138, 'YOSE', 'CA', 761266, 37.83, -119.5, 'Vascular Plant', 'Klamath Plum', None)


In [14]:
# Define column headers that match the SQL SELECT statement above
# This will be used to create a DataFrame with proper column names
original_columns = [
    'scientific_name', 'park_name', 'observations',
    'park_code', 'State', 'Acres', 'Latitude', 'Longitude',
    'category', 'common_names', 'conservation_status'
]

In [15]:
# Convert the list of tuples into a Pandas DataFrame using the defined column names
df = pd.DataFrame(final_joined_data, columns=original_columns)

In [16]:
# Reorder the columns to improve readability or match project requirements
# This groups park information, species details, and observations more logically
new_order = [
    'park_name', 'park_code', 'State', 'Latitude', 'Longitude', 'Acres',
    'category', 'common_names', 'scientific_name', 'observations', 'conservation_status'
]

df = df[new_order]  # Apply the new column order

In [17]:
# Save the final merged dataset to a CSV file in the 'resources' folder
# This file can be used for further analysis, visualizations, or project presentations
df.to_csv("resources/National_Parks_Dataset.csv", index=False)

In [18]:
# Close the connection to the PostgreSQL database once all queries are complete
# This is a good practice to free up system resources and avoid leaving open connections
conn.close()